# Model Report — Training Curves and Statistics

This notebook summarizes the trained model (`models/best_model.pt`), loads metrics from `logs/`, plots training curves, and displays the confusion matrix if available.

In [3]:
# 1) Configure Paths and Run Context
from pathlib import Path
import sys
import importlib

# Resolve project root (this notebook is under notebooks/)
PROJECT_ROOT = Path.cwd().resolve().parents[0] if (Path.cwd().name == 'notebooks') else Path.cwd().resolve()
SRC_DIR = PROJECT_ROOT / 'src'

# Ensure both src/ and project root are importable
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Import config and derive key paths
import config
importlib.reload(config)
CKPT_PATH = config.MODELS_DIR / 'best_model.pt'
LOGS_DIR = config.LOGS_DIR
OUT_DIR = LOGS_DIR / 'report'
OUT_DIR.mkdir(parents=True, exist_ok=True)

print('Project root:', PROJECT_ROOT)
print('Checkpoint:', CKPT_PATH)
print('Logs dir:', LOGS_DIR)
print('Report out dir:', OUT_DIR)

Project root: C:\Users\amant\Documents\aaa-COLLEGE\aaa-semester 5\deep-learning-lab-aman\PROJECT 1\Music-classification-with-FMA-MEDIUM
Checkpoint: C:\Users\amant\Documents\aaa-COLLEGE\aaa-semester 5\deep-learning-lab-aman\PROJECT 1\Music-classification-with-FMA-MEDIUM\models\best_model.pt
Logs dir: C:\Users\amant\Documents\aaa-COLLEGE\aaa-semester 5\deep-learning-lab-aman\PROJECT 1\Music-classification-with-FMA-MEDIUM\logs
Report out dir: C:\Users\amant\Documents\aaa-COLLEGE\aaa-semester 5\deep-learning-lab-aman\PROJECT 1\Music-classification-with-FMA-MEDIUM\logs\report


In [4]:
# 2) Install and Import Dependencies
import math
import os
import json
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid', context='notebook')
np.random.seed(42)

print('Torch:', torch.__version__)


Torch: 2.1.0+cpu


In [5]:
# 3) Load best_model.pt Checkpoint
ckpt = None
if CKPT_PATH.exists():
    ckpt = torch.load(CKPT_PATH, map_location='cpu')
    if isinstance(ckpt, dict):
        print('Checkpoint keys:', list(ckpt.keys())[:10])
    else:
        print('Checkpoint is a state_dict-like object')
else:
    print(f'⚠ Checkpoint not found at {CKPT_PATH}')

Checkpoint keys: ['epoch', 'model_state_dict', 'optimizer_state_dict', 'val_acc', 'val_loss']


In [6]:
# 4) Summarize Model and Parameter Count
from typing import Any
param_count = None
file_size_mb = None
dtype_counts = {}

if CKPT_PATH.exists():
    try:
        from model import create_model
        # Recreate model according to config
        model = create_model(n_mels=config.MODEL_CONFIG['input_shape'][0],
                             n_classes=config.MODEL_CONFIG['n_classes'])
        sd = ckpt.get('model_state_dict', ckpt if isinstance(ckpt, dict) else {})
        if isinstance(sd, dict):
            try: model.load_state_dict(sd, strict=False)
            except Exception as e: print('Load state_dict warning:', e)
        param_count = sum(p.numel() for p in model.parameters())
        print('Model parameters:', f'{param_count:,}')
    except Exception as e:
        # Fallback: count parameters from state_dict tensors
        sd = ckpt.get('model_state_dict', ckpt if isinstance(ckpt, dict) else {})
        if isinstance(sd, dict):
            param_count = int(sum(v.numel() for v in sd.values()))
            print('Parameter count (state_dict sum):', f'{param_count:,}')
        else:
            print('Could not infer parameter count.')
    try:
        file_size_mb = os.path.getsize(CKPT_PATH) / (1024*1024)
        print('Checkpoint size (MB):', f'{file_size_mb:.2f}')
    except Exception:
        pass
    # Dtype distribution
    try:
        sd = ckpt.get('model_state_dict', ckpt if isinstance(ckpt, dict) else {})
        for k,v in sd.items():
            dt = str(getattr(v, 'dtype', 'unknown'))
            dtype_counts[dt] = dtype_counts.get(dt, 0) + 1
        print('Dtype counts:', dtype_counts)
    except Exception:
        pass

Model parameters: 423,760
Checkpoint size (MB): 4.88
Dtype counts: {'torch.float32': 28, 'torch.int64': 4}


In [7]:
# 5) Extract Training Metadata from Checkpoint
import pandas as pd
meta_fields = ['epoch','val_acc','val_loss','best_metric','best_epoch','train_time','git_sha']
rows = {}
if isinstance(ckpt, dict):
    for k in meta_fields:
        if k in ckpt:
            rows[k] = ckpt[k]
if rows:
    display(pd.DataFrame([rows]))
else:
    print('No training metadata found in checkpoint.')

,epoch,val_acc,val_loss
0,0,62.553062,1.301814


In [17]:
# 6) Load Metrics: TensorBoard Event Files (optional)
from glob import glob
tb_df = None
try:
    from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
    event_files = glob(str(LOGS_DIR / '**' / 'events.out.tfevents.*'), recursive=True)
    records = []
    for ef in event_files:
        try:
            ea = EventAccumulator(ef)
            ea.Reload()
            for tag in ['train/loss','val/loss','train/acc','val/acc','lr']:
                if tag in ea.Tags().get('scalars', []):
                    for w in ea.Scalars(tag):
                        records.append({'run': os.path.basename(os.path.dirname(ef)), 'tag': tag, 'step': w.step, 'wall_time': w.wall_time, 'value': w.value})
        except Exception:
            pass
    if records:
        tb_df = pd.DataFrame(records)
        print('Loaded TensorBoard scalars:', len(tb_df))
    else:
        print('No TensorBoard scalars found.')
except Exception as e:
    print('TensorBoard not available or failed to load:', e)

No TensorBoard scalars found.


In [18]:
# 7) Load Metrics: history.csv / metrics.csv / metrics.json
hist = None
hist_candidates = [LOGS_DIR / 'training_metrics.csv', LOGS_DIR / 'history.csv', PROJECT_ROOT / 'history.csv']
for p in hist_candidates:
    if p.exists():
        try:
            hist = pd.read_csv(p)
            print('Loaded metrics table from', p)
            break
        except Exception:
            pass
metrics_json = None
metrics_json_path = LOGS_DIR / 'metrics.json'
if metrics_json_path.exists():
    try:
        with open(metrics_json_path, 'r') as f:
            metrics_json = json.load(f)
        print('Loaded evaluation metrics from', metrics_json_path)
    except Exception:
        pass

# Normalize column names
if hist is not None:
    cols = {c.lower().strip(): c for c in hist.columns}
    rename = {}
    for key in ['epoch','step','train_loss','val_loss','train_acc','val_acc','lr']:
        if key in cols:
            rename[cols[key]] = key
    hist = hist.rename(columns=rename)
    if 'epoch' not in hist.columns and 'step' in hist.columns:
        hist['epoch'] = hist['step']
    if 'step' not in hist.columns and 'epoch' in hist.columns:
        hist['step'] = hist['epoch']

In [19]:
# 8) Merge and Clean Metrics (+ EMA smoothing)
merged = None
if hist is not None and not hist.empty:
    merged = hist.copy()
    merged = merged.sort_values(by=['epoch']).reset_index(drop=True)
    # EMA smoothing for curves
    def ema(series, alpha=0.2):
        out = []
        prev = None
        for x in series:
            prev = (alpha * x + (1-alpha) * prev) if prev is not None else x
            out.append(prev)
        return out
    for col in ['train_loss','val_loss','train_acc','val_acc']:
        if col in merged.columns:
            merged[f'{col}_ema'] = ema(merged[col].values, alpha=0.2)
    display(merged.head())
else:
    print('No history table available for merge.')

No history table available for merge.


In [11]:
# 9) Plot Training/Validation Loss and Accuracy
if merged is not None and not merged.empty:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))
    x = merged['epoch'] if 'epoch' in merged else range(len(merged))
    # Loss
    if 'train_loss' in merged: ax1.plot(x, merged['train_loss'], label='train_loss', alpha=0.6)
    if 'val_loss' in merged: ax1.plot(x, merged['val_loss'], label='val_loss', alpha=0.8)
    if 'train_loss_ema' in merged: ax1.plot(x, merged['train_loss_ema'], label='train_loss_ema', lw=2)
    if 'val_loss_ema' in merged: ax1.plot(x, merged['val_loss_ema'], label='val_loss_ema', lw=2)
    ax1.set_title('Loss'); ax1.set_xlabel('Epoch'); ax1.set_ylabel('Loss'); ax1.legend(); ax1.grid(True, alpha=0.3)
    # Accuracy
    if 'train_acc' in merged: ax2.plot(x, merged['train_acc'], label='train_acc', alpha=0.6)
    if 'val_acc' in merged: ax2.plot(x, merged['val_acc'], label='val_acc', alpha=0.8)
    if 'train_acc_ema' in merged: ax2.plot(x, merged['train_acc_ema'], label='train_acc_ema', lw=2)
    if 'val_acc_ema' in merged: ax2.plot(x, merged['val_acc_ema'], label='val_acc_ema', lw=2)
    ax2.set_title('Accuracy (%)'); ax2.set_xlabel('Epoch'); ax2.set_ylabel('Accuracy'); ax2.legend(); ax2.grid(True, alpha=0.3)
    plt.tight_layout()
    fig.savefig(OUT_DIR / 'training_curves.png', dpi=200)
    plt.show()
    print('Saved:', OUT_DIR / 'training_curves.png')
else:
    print('No merged metrics to plot.')

No merged metrics to plot.


In [12]:
# 10) Identify Best Epoch and Annotate
best_epoch = None
text = ''
if merged is not None and not merged.empty:
    if 'val_loss' in merged:
        best_epoch = int(merged['val_loss'].idxmin()) + 1
        text = f'Best by val_loss: epoch {best_epoch}, val_loss={merged.loc[best_epoch-1, "val_loss"]:.4f}'
    elif 'val_acc' in merged:
        best_epoch = int(merged['val_acc'].idxmax()) + 1
        text = f'Best by val_acc: epoch {best_epoch}, val_acc={merged.loc[best_epoch-1, "val_acc"]:.2f}%'
    print(text)
else:
    print('No metrics to identify best epoch.')

No metrics to identify best epoch.


In [13]:
# 11) Additional Stats and Learning Rate Schedule
if merged is not None and not merged.empty and 'lr' in merged:
    plt.figure(figsize=(6,4))
    plt.plot(merged['epoch'], merged['lr'], label='lr')
    plt.xlabel('Epoch'); plt.ylabel('Learning Rate'); plt.title('LR Schedule')
    plt.grid(True, alpha=0.3); plt.legend();
    plt.tight_layout()
    plt.savefig(OUT_DIR / 'lr_schedule.png', dpi=200)
    plt.show()
    print('Saved:', OUT_DIR / 'lr_schedule.png')
else:
    print('LR not available.')

LR not available.


In [14]:
# 12) Optional: Re-evaluate Best Model on Validation Set
print('Optional: Re-evaluation placeholder. Use evaluate.py for full evaluation.')

Optional: Re-evaluation placeholder. Use evaluate.py for full evaluation.


In [15]:
# 13) Optional: Confusion Matrix and ROC/PR Curves
from IPython.display import Image as IPyImage, display
cm_candidates = [
    LOGS_DIR / 'confusion_matrix.png',
    LOGS_DIR / 'confusion-matrix.png',
    LOGS_DIR / 'confusion_matrix.jpg',
    LOGS_DIR / 'confusion-matrix.jpg',
]
cm_path = next((p for p in cm_candidates if p.exists()), None)
if cm_path:
    print('Confusion Matrix:')
    display(IPyImage(filename=str(cm_path)))
else:
    print('No confusion matrix found in logs/.')

No confusion matrix found in logs/.


In [16]:
# 14) Save Figures and Export Report
# Save merged metrics to CSV and write a small summary JSON
if merged is not None and not merged.empty:
    merged.to_csv(OUT_DIR / 'merged_metrics.csv', index=False)
    summary = {
        'checkpoint': str(CKPT_PATH),
        'params': int(param_count) if param_count is not None else None,
        'checkpoint_size_mb': float(f'{(os.path.getsize(CKPT_PATH)/(1024*1024)):.4f}') if CKPT_PATH.exists() else None,
        'best_epoch_text': text,
    }
    with open(OUT_DIR / 'summary.json', 'w') as f:
        json.dump(summary, f, indent=2)
    print('Saved:', OUT_DIR / 'merged_metrics.csv')
    print('Saved:', OUT_DIR / 'summary.json')
else:
    print('No merged metrics to export.')

No merged metrics to export.
